**FIFA PLAYER PREDICTION**

This is a Mid-semester project created by Paa Kwesi Atobrah and Samuel Dartey-Baah which predicts a players overall rating

In [ ]:
#importing all the necessary libraries which would be used
import pandas as pd
import numpy as np
import os
import sklearn
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#reading the data set which would be used for training model and testing
player_21_df = pd.read_csv('/content/drive/My Drive/players_21.csv')
player_22_df = pd.read_csv('/content/drive/My Drive/players_22.csv')

<ipython-input-37-7bce5034312a>:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  player_22_df = pd.read_csv('/content/drive/My Drive/players_22.csv')


**DATA PREPARATIONS AND FEATURE EXTRACTION**

At this stage we remove all useless values, impute necessary values and choose all the important features need to predict the overall of a player

In [ ]:
#looking at the first 5 tuples of player21 dataset
player_21_df.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [ ]:
#looking at descriptive statistics of the data
player_21_df.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,18944.000000,18944.000000,18944.000000,1.870700e+04,18719.000000,18944.000000,18944.000000,18944.000000,18719.000000,18719.000000,...,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,2083.000000
mean,226232.328917,65.677787,71.086729,2.902288e+06,9148.482825,25.225823,181.190773,75.016892,47908.905551,1.355468,...,57.978674,46.470017,47.581767,45.546505,16.446052,16.236486,16.103357,16.225982,16.551309,37.203553
std,27166.574284,7.002278,6.109985,7.743775e+06,19893.149956,4.697354,6.825672,7.057140,53906.733776,0.739015,...,12.118390,20.169591,21.402461,20.953997,17.577332,16.845480,16.519399,17.017341,17.878121,10.714523
min,41.000000,47.000000,47.000000,9.000000e+03,500.000000,16.000000,155.000000,50.000000,1.000000,1.000000,...,12.000000,3.000000,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,210028.750000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,461.000000,1.000000,...,50.000000,29.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000,28.000000
50%,232301.500000,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,181.000000,75.000000,1913.000000,1.000000,...,59.000000,52.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000,37.000000
75%,246745.750000,70.000000,75.000000,2.000000e+06,8000.000000,29.000000,186.000000,80.000000,110986.000000,1.000000,...,66.000000,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000,45.000000
max,258970.000000,93.000000,95.000000,1.855000e+08,560000.000000,53.000000,206.000000,110.000000,114899.000000,4.000000,...,96.000000,94.000000,93.000000,90.000000,90.000000,92.000000,93.000000,91.000000,90.000000,65.000000


In [ ]:
#Creating a new dataset
cleaned_player_21_df = player_21_df
cleaned_player_22_df = player_22_df

In [ ]:
#Dropping unnecessary columns(URLs)
cleaned_player_21_df.drop(columns=['player_url', 'player_face_url', 'club_logo_url', 'club_flag_url', 'nation_logo_url', 'nation_flag_url'], inplace=True)
cleaned_player_22_df.drop(columns=['player_url', 'player_face_url', 'club_logo_url', 'club_flag_url', 'nation_logo_url', 'nation_flag_url'], inplace=True)


#Dropping columns with more than 30% of their values missing
player_21_df_size = cleaned_player_21_df.shape[0]
cleaned_player_21_df = cleaned_player_21_df.dropna(axis = 1, thresh = (player_21_df_size * 0.3))

player_22_df_size = cleaned_player_22_df.shape[0]
cleaned_player_22_df = cleaned_player_22_df.dropna(axis = 1, thresh = (player_22_df_size * 0.3))



print(f'This reduces the number of columns in player 21 from {player_21_df.shape[1]} to {cleaned_player_21_df.shape[1]}')

print(f'This reduces the number of columns in player 22 from {player_22_df.shape[1]} to {cleaned_player_22_df.shape[1]}')


This reduces the number of columns in player 21 from 104 to 98
This reduces the number of columns in player 22 from 104 to 98


In [ ]:
#Grouping the different positions
#defining the mapping between the positions
position_mapping = {'LS': 'Forward', 'ST': 'Forward', 'RS': 'Forward', 'LW': 'Forward','LF': 'Forward','CF': 'Forward','RF': 'Forward','RW': 'Forward',
                    'LAM': 'Midfield','CAM': 'Midfield','RAM': 'Midfield','LM': 'Midfield','LCM': 'Midfield','CM': 'Midfield','RCM': 'Midfield','RM': 'Midfield',
                    'LWB': 'Defender','LDM': 'Defender','CDM': 'Defender','RDM': 'Defender','RWB': 'Defender','LB': 'Defender','LCB': 'Defender','CB': 'Defender','RCB': 'Defender','RB': 'Defender',
                    'GK': 'GoalKeeping'}

#replacing the different positions with the general positions
cleaned_player_21_df['player_positions'] = cleaned_player_21_df['player_positions'].str.split(', ').str[0] #spliting the positions so they can be replaced
cleaned_player_21_df['player_positions'] = cleaned_player_21_df['player_positions'].replace(position_mapping)


cleaned_player_22_df['player_positions'] = cleaned_player_22_df['player_positions'].str.split(', ').str[0]
cleaned_player_22_df['player_positions'] = cleaned_player_22_df['player_positions'].replace(position_mapping)


In [ ]:
#data set with positions changed with general positions
cleaned_player_21_df.head()


,sofifa_id,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023,L. Messi,Lionel Andrés Messi Cuccittini,Forward,93,93,103500000.0,560000.0,33,1987-06-24,...,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3,19+3
1,20801,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,Forward,92,92,63000000.0,220000.0,35,1985-02-05,...,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3,20+3
2,188545,R. Lewandowski,Robert Lewandowski,Forward,91,91,111000000.0,240000.0,31,1988-08-21,...,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
3,190871,Neymar Jr,Neymar da Silva Santos Júnior,Forward,91,91,132000000.0,270000.0,28,1992-02-05,...,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3,20+3
4,192985,K. De Bruyne,Kevin De Bruyne,Midfield,91,91,129000000.0,370000.0,29,1991-06-28,...,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3


In [ ]:
#Label encoding the general positions and dropping player_position
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
cleaned_player_21_df['player_positions' + '_encoded'] = label_encoder.fit_transform(cleaned_player_21_df['player_positions'])
cleaned_player_21_df.drop(columns=['player_positions'], inplace = True)

label_encoder2 = LabelEncoder()
cleaned_player_22_df['player_positions' + '_encoded'] = label_encoder2.fit_transform(cleaned_player_22_df['player_positions'])
cleaned_player_22_df.drop(columns=['player_positions'], inplace = True)


In [ ]:
#player_position_encoded added to the dataset
cleaned_player_21_df.head()

,sofifa_id,short_name,long_name,overall,potential,value_eur,wage_eur,age,dob,height_cm,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_positions_encoded
0,158023,L. Messi,Lionel Andrés Messi Cuccittini,93,93,103500000.0,560000.0,33,1987-06-24,170,...,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3,19+3,1
1,20801,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,92,92,63000000.0,220000.0,35,1985-02-05,187,...,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3,20+3,1
2,188545,R. Lewandowski,Robert Lewandowski,91,91,111000000.0,240000.0,31,1988-08-21,184,...,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,1
3,190871,Neymar Jr,Neymar da Silva Santos Júnior,91,91,132000000.0,270000.0,28,1992-02-05,175,...,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3,20+3,1
4,192985,K. De Bruyne,Kevin De Bruyne,91,91,129000000.0,370000.0,29,1991-06-28,181,...,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3,3


**FEATURE SUBSETTING FOR MAXIMUM CORRELATION**

At this step we will select the most important/relevant features needed for the machine learning process, dropping all useless columns

In [ ]:
#removing certain columns which are not necessary in training the model
new_cleaned_player_21_df = cleaned_player_21_df
new_cleaned_player_22_df = cleaned_player_22_df

new_cleaned_player_21_df.drop(columns=['short_name', 'long_name', 'dob','preferred_foot', 'club_name', 'league_name', 'weight_kg' ,'height_cm','club_position', 'club_jersey_number', 'club_joined', 'club_contract_valid_until', 'nationality_name', 'work_rate','wage_eur', 'body_type', 'real_face', 'player_traits','nationality_id', 'club_jersey_number', 'club_team_id','league_level','sofifa_id'], inplace=True)
new_cleaned_player_22_df.drop(columns=['short_name', 'long_name', 'dob','preferred_foot', 'club_name', 'league_name','weight_kg' ,'height_cm', 'club_position', 'club_jersey_number', 'club_joined', 'club_contract_valid_until', 'nationality_name', 'work_rate','wage_eur', 'body_type', 'real_face', 'player_traits','nationality_id', 'club_jersey_number', 'club_team_id','league_level','sofifa_id'], inplace=True)



In [ ]:
new_cleaned_player_21_df.head()


,overall,potential,value_eur,age,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_positions_encoded
0,93,93,103500000.0,33,4,4,5,138400000.0,85.0,92.0,...,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3,19+3,1
1,92,92,63000000.0,35,4,5,5,75900000.0,89.0,93.0,...,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3,20+3,1
2,91,91,111000000.0,31,4,4,4,132000000.0,78.0,91.0,...,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,1
3,91,91,132000000.0,28,5,5,5,166500000.0,91.0,85.0,...,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3,20+3,1
4,91,91,129000000.0,29,5,4,4,161000000.0,76.0,86.0,...,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3,3


In [ ]:
#label encoding the ls, st and the other positions so they can be used for training the model
from sklearn.preprocessing import LabelEncoder

label_encoder_player_positions = LabelEncoder()
new_cleaned_player_21_df['ls' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['ls'])
new_cleaned_player_21_df['st' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['st'])
new_cleaned_player_21_df['rs' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['rs'])
new_cleaned_player_21_df['lw' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['lw'])
new_cleaned_player_21_df['lf' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['lf'])
new_cleaned_player_21_df['cf' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['cf'])
new_cleaned_player_21_df['rf' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['rf'])
new_cleaned_player_21_df['rw' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['rw'])
new_cleaned_player_21_df['lam' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['lam'])
new_cleaned_player_21_df['cam' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['cam'])
new_cleaned_player_21_df['ram' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['ram'])
new_cleaned_player_21_df['lm' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['lm'])
new_cleaned_player_21_df['lcm' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['lcm'])
new_cleaned_player_21_df['cm' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['cm'])
new_cleaned_player_21_df['rcm' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['rcm'])
new_cleaned_player_21_df['rm' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['rm'])
new_cleaned_player_21_df['lwb' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['lwb'])
new_cleaned_player_21_df['ldm' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['ldm'])
new_cleaned_player_21_df['cdm' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['cdm'])
new_cleaned_player_21_df['rdm' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['rdm'])
new_cleaned_player_21_df['rwb' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['rwb'])
new_cleaned_player_21_df['lb' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['lb'])
new_cleaned_player_21_df['lcb' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['lcb'])
new_cleaned_player_21_df['cb' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['cb'])
new_cleaned_player_21_df['rcb' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['rcb'])
new_cleaned_player_21_df['rb' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['rb'])
new_cleaned_player_21_df['gk' + '_encoded'] = label_encoder_player_positions.fit_transform(new_cleaned_player_21_df['gk'])


new_cleaned_player_21_df.drop(columns=['ls', 'st', 'rs', 'lw','lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm','cm', 'rcm', 'rm', 'lwb','ldm', 'cdm', 'rdm','rwb','lb','lcb','cb','rcb','rb','gk'], inplace=True)

label_encoder_player_positions2 = LabelEncoder()
new_cleaned_player_22_df['ls' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['ls'])
new_cleaned_player_22_df['st' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['st'])
new_cleaned_player_22_df['rs' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['rs'])
new_cleaned_player_22_df['lw' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['lw'])
new_cleaned_player_22_df['lf' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['lf'])
new_cleaned_player_22_df['cf' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['cf'])
new_cleaned_player_22_df['rf' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['rf'])
new_cleaned_player_22_df['rw' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['rw'])
new_cleaned_player_22_df['lam' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['lam'])
new_cleaned_player_22_df['cam' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['cam'])
new_cleaned_player_22_df['ram' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['ram'])
new_cleaned_player_22_df['lm' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['lm'])
new_cleaned_player_22_df['lcm' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['lcm'])
new_cleaned_player_22_df['cm' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['cm'])
new_cleaned_player_22_df['rcm' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['rcm'])
new_cleaned_player_22_df['rm' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['rm'])
new_cleaned_player_22_df['lwb' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['lwb'])
new_cleaned_player_22_df['ldm' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['ldm'])
new_cleaned_player_22_df['cdm' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['cdm'])
new_cleaned_player_22_df['rdm' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['rdm'])
new_cleaned_player_22_df['rwb' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['rwb'])
new_cleaned_player_22_df['lb' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['lb'])
new_cleaned_player_22_df['lcb' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['lcb'])
new_cleaned_player_22_df['cb' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['cb'])
new_cleaned_player_22_df['rcb' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['rcb'])
new_cleaned_player_22_df['rb' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['rb'])
new_cleaned_player_22_df['gk' + '_encoded'] = label_encoder_player_positions2.fit_transform(new_cleaned_player_22_df['gk'])

new_cleaned_player_22_df.drop(columns=['ls', 'st', 'rs', 'lw','lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm','cm', 'rcm', 'rm', 'lwb','ldm', 'cdm', 'rdm','rwb','lb','lcb','cb','rcb','rb','gk'], inplace=True)


new_cleaned_player_21_df.head()




In [ ]:
print(new_cleaned_player_21_df.isnull().sum())
print()
print('There still appears to be null values')

overall          0
potential        0
value_eur      237
age              0
weak_foot        0
              ... 
lcb_encoded      0
cb_encoded       0
rcb_encoded      0
rb_encoded       0
gk_encoded       0
Length: 76, dtype: int64

There still appears to be null values


In [ ]:
#filling the null values in the dataset with the means of each column
from sklearn.impute import SimpleImputer

sc=SimpleImputer(strategy='mean')
new_cleaned_player_21_df = pd.DataFrame(sc.fit_transform(new_cleaned_player_21_df), columns=new_cleaned_player_21_df.columns)
new_cleaned_player_22_df = pd.DataFrame(sc.fit_transform(new_cleaned_player_22_df), columns=new_cleaned_player_22_df.columns)

In [ ]:
print(new_cleaned_player_21_df.isnull().sum())

overall        0
potential      0
value_eur      0
age            0
weak_foot      0
              ..
lcb_encoded    0
cb_encoded     0
rcb_encoded    0
rb_encoded     0
gk_encoded     0
Length: 76, dtype: int64


In [ ]:
#Finding the most important features needed to predict a player's overall with correlation matrix
corr_matrix = new_cleaned_player_21_df.corr()
needed_features=corr_matrix["overall"].sort_values(ascending=False)
needed_features = needed_features[ (needed_features >= 0.4) & ~(needed_features.index.str.contains('encoded')) & ~(needed_features.index.str.contains('overall'))]
needed_features = needed_features.head(10).index.tolist()
print(f'These are the most necessary features: {needed_features}')

These are the most necessary features: ['movement_reactions', 'mentality_composure', 'passing', 'potential', 'release_clause_eur', 'dribbling', 'power_shot_power', 'value_eur', 'mentality_vision', 'attacking_short_passing']


**TRAINING THE DATA **



In [ ]:
#splitting the X and Y set which would be used for training
X = new_cleaned_player_21_df[needed_features]
Y = new_cleaned_player_21_df['overall']

In [ ]:
#scaling the X data
from sklearn.preprocessing import StandardScaler
StandardScaler = StandardScaler()

X = StandardScaler.fit_transform(X.copy())
X = pd.DataFrame(X, columns = needed_features)


In [ ]:
#Random Forest Regressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix, classification_report, r2_score

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model_rf = RandomForestRegressor(n_estimators=500,random_state=42)
model_rf.fit(X_train, Y_train)


RandomForestRegressor(n_estimators=500, random_state=42)

In [ ]:
# Cross-validation
cross_val_score(model_rf, X_test, Y_test, cv=5)

array([0.94980692, 0.96161301, 0.96293588, 0.96340699, 0.96574802])

In [ ]:
#measuring the performance of random Forest Regressor Model
model_rf_predicted = model_rf.predict(X_test)
model_rf_mae = mean_absolute_error(Y_test, model_rf_predicted)
model_rf_mse = mean_squared_error(Y_test, model_rf_predicted)
model_rf_r2 = r2_score(Y_test, model_rf_predicted)

In [ ]:
print(f'Mean Absolute Error :{model_rf_mae}')
print(f'Mean Squared Error :{model_rf_mse}')
print(f'R-square :{model_rf_r2}')

Mean Absolute Error :0.624469253101082
Mean Squared Error :1.1476655845869623
R-square :0.9756515070676395


In [ ]:
#XGBoost
import xgboost as xgb

model_xgb = xgb.XGBRegressor()
model_xgb.fit(X_train, Y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
# Cross-validation
cross_val_score(model_xgb, X_test, Y_test, cv=5)

array([0.95435216, 0.96279158, 0.95909707, 0.96288503, 0.96633524])

In [ ]:
#measuring the performance of the XGBoost model
model_xgb_predicted = model_xgb.predict(X_test)
model_xgb_mae = mean_absolute_error(Y_test, model_xgb_predicted)
model_xgb_mse = mean_squared_error(Y_test, model_xgb_predicted)
model_xgb_r2 = r2_score(Y_test, model_xgb_predicted)

In [ ]:
print(f'Mean Absolute Error :{model_xgb_mae}')
print(f'Mean Squared Error :{model_xgb_mse}')
print(f'R-square :{model_xgb_r2}')

Mean Absolute Error :0.7675440756303712
Mean Squared Error :1.2758700604701383
R-square :0.9729315633690037


In [ ]:
#Gradient Boost Regressor
from sklearn.ensemble import GradientBoostingRegressor

# Create and train the Gradient Boosting model
model_gb = GradientBoostingRegressor(n_estimators=500, random_state=42)
model_gb.fit(X_train, Y_train)

GradientBoostingRegressor(n_estimators=500, random_state=42)

In [ ]:
# Cross-validation
cross_val_scores = cross_val_score(model_gb, X_test, Y_test, cv=5)

In [ ]:
# Measuring the performance of the Gradient Boosting Model
model_gb_predicted = model_gb.predict(X_test)
model_gb_mae = mean_absolute_error(Y_test, model_gb_predicted)
model_gb_mse = mean_squared_error(Y_test, model_gb_predicted)
model_gb_r2 = r2_score(Y_test, model_gb_predicted)

In [ ]:
print(f"Mean Absolute Error: {model_gb_mae}")
print(f"Mean Squared Error: {model_gb_mse}")
print(f"R-squared: {model_gb_r2}")

Mean Absolute Error: 0.8690014804812031
Mean Squared Error: 1.5366212951232536
R-squared: 0.9673995515361835


In [ ]:
#fine tuning the model
# For Random Forest
param_grid_rf = {'n_estimators': [500, 1000], 'max_depth': [ 10, 12, 15]}
grid_rf = GridSearchCV(model_rf, param_grid_rf, cv=5)
grid_rf.fit(X_train, Y_train)

# For XGBoost
param_grid_xgb = {'n_estimators': [500, 1000], 'learning_rate': [0.01, 0.1, 0.2]}
grid_xgb = GridSearchCV(model_xgb, param_grid_xgb, cv=5)
grid_xgb.fit(X_train, Y_train)

# for Gradient Boost Regressor
param_grid_gb = {'n_estimators': [500, 1000],'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7], 'subsample': [0.8, 0.9, 1.0],}
grid_gb = GridSearchCV(model_gb, param_grid_gb, cv=5)
grid_gb.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=GradientBoostingRegressor(n_estimators=500,
                                                 random_state=42),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'n_estimators': [500, 1000],
                         'subsample': [0.8, 0.9, 1.0]})

In [ ]:
model_pred_classes_rf = np.round(model_rf_predicted).astype(int)
model_pred_classes_xgb = np.round(model_xgb_predicted).astype(int)
model_pred_classes_gb = np.round(model_gb_predicted).astype(int)


# Confusion Matrix
print(confusion_matrix(Y_test, model_pred_classes_rf))
print(confusion_matrix(Y_test, model_pred_classes_xgb))
print(confusion_matrix(Y_test, model_pred_classes_gb))

[[0 0 2 ... 0 0 0]
 [0 0 4 ... 0 0 0]
 [0 0 3 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


In [ ]:
# Classification Report
print(classification_report(Y_test, model_pred_classes_rf))
print(classification_report(Y_test, model_pred_classes_xgb))
print(classification_report(Y_test, model_pred_classes_gb))

              precision    recall  f1-score   support

        47.0       0.00      0.00      0.00         3
        48.0       0.00      0.00      0.00        11
        49.0       0.16      0.38      0.22         8
        50.0       0.37      0.43      0.40        23
        51.0       0.92      0.62      0.74        37
        52.0       0.71      0.63      0.67        46
        53.0       0.60      0.71      0.65        41
        54.0       0.51      0.61      0.56        36
        55.0       0.81      0.53      0.64        66
        56.0       0.52      0.63      0.57        63
        57.0       0.68      0.68      0.68        96
        58.0       0.67      0.56      0.61       105
        59.0       0.51      0.58      0.54       107
        60.0       0.64      0.66      0.65       161
        61.0       0.64      0.58      0.61       148
        62.0       0.69      0.68      0.69       210
        63.0       0.64      0.61      0.62       233
        64.0       0.59    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Testing The Model With A New Dataset**

In [134]:
# splitting X and Y for player 22 dataset
X = new_cleaned_player_22_df[needed_features]
Y = new_cleaned_player_22_df['overall']

# scale the X
from sklearn.preprocessing import StandardScaler
StandardScaler = StandardScaler()

X = StandardScaler.fit_transform(X.copy())
X = pd.DataFrame(X, columns = needed_features)

In [136]:
#testing player 22 dataset with Random Forest


model_rf.fit(X_test, Y_test)
#measuring the performance
model_rf_predicted = model_rf.predict(X_test)
model_rf_mae = mean_absolute_error(Y_test, model_rf_predicted)

print(f"Mean Absolute Error: {model_rf_mae}")

Mean Absolute Error: 0.29099973607812096


In [137]:
#testing player 22 dataset with XGBoost

model_xgb.fit(X_test, Y_test)
#measuring the performance
model_xgb_predicted = model_xgb.predict(X_test)
model_xgb_mae = mean_absolute_error(Y_test, model_xgb_predicted)
cross_val_score(model_xgb, X_test, Y_test, cv=5)

print(f"Mean Absolute Error: {model_xgb_mae}")

Mean Absolute Error: 0.29384775531735574


In [140]:
#testing player 22 dataset with Gradient Boost

model_gb.fit(X_test, Y_test)
#measuring the performance
model_gb_predicted = model_gb.predict(X_test)
model_gb_mae = mean_absolute_error(Y_test, model_gb_predicted)
cross_val_score(model_gb, X_test, Y_test, cv=5)

print(f"Mean Absolute Error: {model_gb_mae}")

Mean Absolute Error: 0.6675189541914206


**Deploying The Model**

Saving the model with the best mean absolute mean which is the random Forest

In [141]:
#Saving the model using pickle
import pickle
import joblib

#save the random forest model
with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(model_rf, f)


In [142]:
import joblib

joblib.dump(model_rf, '/content/random_forest_model.pkl')

['/content/random_forest_model.pkl']